# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    sum = 0
    for n in graph.nodes():
        sum = sum + graph.node[n]['demand']
    if sum != 0:
        raise nx.NetworkXUnfeasible("flow does not satisfy demands")
    else:
        graph.add_node('S')
        graph.node['S']['demand'] = 0
        graph.add_node('T')
        graph.node['T']['demand'] = 0
        for state in graph.nodes():
            if graph.node[state]['demand'] < 0:
                graph.add_edge('S',state)
                graph.edge['S'][state]['capacity'] = abs(graph.node[state]['demand'])
            if graph.node[state]['demand'] > 0:
                graph.add_edge(state,'T')
                graph.edge[state]['T']['capacity'] = abs(graph.node[state]['demand'])


        def checkpath(graph_res):
            visited = []
            visited.append('S')
            graph_res.node['S']['parent'] = "nil"
            q = []
            q.append("S")
            while len(q)>0:
                e = q.pop(0)
                for u in graph_res.neighbors(e):
                    if u not in visited:
                        visited.append(u)
                        graph_res.node[u]['parent']=e
                        q.append(u)
            if 'T' in visited:
                return True
            else:
                return False

        def getpath(n,mypath,graph_res):
            if graph_res.node[n]['parent'] != "nil":
                parent=graph_res.node[n]['parent']
                mypath.append(parent)
                getpath(parent,mypath,graph_res)
            else:
                mypath.reverse()
            return mypath

        def initiateflow():
            for (s1,s2) in graph.edges():
                graph[s1][s2]['flow']=0

        def getminflow(graph_res, traversed):
            minlist = []
            for one,two in zip(traversed, traversed[1:]):
                minlist.append(graph_res[one][two]['capacity'])
            mincapacity = min(minlist)
            return mincapacity

        def initiateres(mygraph):
            graph_res = mygraph.copy()
            return graph_res

        def updategraph(mygraph,f,path):
            for one,two in zip(path, path[1:]):
                mygraph[one][two]['flow'] = mygraph[one][two]['flow']+f
            return graph

        def updateres(graph_res,f,path):
            for one,two in zip(path, path[1:]):
                graph_res[one][two]['capacity']=graph_res[one][two]['capacity']-f
                if graph_res.has_edge(two,one):
                    graph_res[two][one]['capacity']=graph_res[two][one]['capacity']+f
                else:
                    graph_res.add_edge(two,one,capacity=f)
            for s1,s2 in graph_res.edges():
                if graph_res[s1][s2]['capacity']==0:
                    graph_res.remove_edge(s1,s2) 
            return graph_res

        initiateflow()
        graph_res = initiateres(graph)
        maxflow = 0
        while checkpath(graph_res) == True:
            mypath = ['T']
            path = getpath('T',mypath,graph_res)
            flow = getminflow(graph_res,path)
            maxflow = maxflow+flow
            graph = updategraph(graph,flow,path)
            graph_res = updateres(graph_res,flow,path)
            
        dest_sum=0
        src_sum=0
        for (s1,s2) in graph.edges():
            if (s2=='T'):
                dest_sum=dest_sum+graph.edge[s1][s2]['capacity']
            if (s1=='S'):
                src_sum=src_sum+graph.edge[s1][s2]['capacity']

        if ((maxflow != src_sum) | (maxflow != dest_sum)):
            raise nx.NetworkXUnfeasible("flow does not satisfy demands")

        final_dict = {}
        internal_dict = {}
        for n in graph.nodes():
            final_dict[n] = []

        for n in graph.nodes():
            for neigh in graph.neighbors(n):
                internal_dict[neigh]=graph[n][neigh]['flow']
            final_dict[n]=internal_dict
            internal_dict = {}

        graph.remove_node('S')
        graph.remove_node('T')

        if 'S' in final_dict:
            del final_dict['S']
        if 'T' in final_dict:
            del final_dict['T']

        for (key,values) in final_dict.items():
            if 'S' in values:
                del values['S']
            if 'T' in values:
                del values['T']
        return final_dict

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    outflow = {}
    for (key,values) in flow.items():
        sum = 0
        for (k,v) in values.items():
            sum = sum + v
        outflow.update({key:sum})
        
    inflow = {}
    for (key,values) in flow.items():
        for (k,v) in values.items():
            if k in inflow:
                inflow[k] = inflow[k]+v
            else:
                inflow.update({k:v})
    div = {key: inflow[key] - outflow.get(key, 0) for key in inflow.keys()}
    return div

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

NetworkXUnfeasible: flow does not satisfy demands